In [5]:
import pandas as pd
import numpy as np
import torch
import csv
from util_v3 import *
from data_v3 import load_data, get_train_datasets
from model_v3 import *
from torch.utils.data import dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable

# from mscn.data import load_data

import torch
import torch.nn as nn
import torch.nn.functional as F
# from mscn.data import get_train_datasets, load_data, make_dataset

## load and encode train data by calling the function

In [6]:
dataset_name = 'tpcds'

dicts, column_min_max_vals, labels_train, labels_test, max_num_joins, max_num_predicates, train_data, test_data = get_train_datasets(dataset_name)

Loaded queries
Number of training samples: 9336
Number of validation samples: 1038
Created TensorDataset for training data
Created TensorDataset for validation data


In [7]:
num_epochs = 10
batch_size = 1
hid_units = 256
cuda = False

num_materialized_samples = 0

num_classes = 19

In [8]:
table2vec, column2vec, op2vec, join2vec = dicts

num_materialized_samples = 0

# Train model
sample_feats = len(table2vec) + num_materialized_samples
predicate_feats = len(column2vec) + len(op2vec) + 1
join_feats = len(join2vec)

model = SetConv(sample_feats, predicate_feats, join_feats, hid_units, num_classes)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_data_loader = DataLoader(train_data, batch_size=batch_size)
# test_data_loader = DataLoader(test_data, batch_size=batch_size)

model.train()

SetConv(
  (sample_mlp1): Linear(in_features=26, out_features=256, bias=True)
  (sample_mlp2): Linear(in_features=256, out_features=256, bias=True)
  (predicate_mlp1): Linear(in_features=30, out_features=256, bias=True)
  (predicate_mlp2): Linear(in_features=256, out_features=256, bias=True)
  (join_mlp1): Linear(in_features=56, out_features=256, bias=True)
  (join_mlp2): Linear(in_features=256, out_features=256, bias=True)
  (out_mlp1): Linear(in_features=768, out_features=256, bias=True)
  (out_mlp2): Linear(in_features=256, out_features=19, bias=True)
)

In [9]:
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    loss_total = 0.

    for batch_idx, data_batch in enumerate(train_data_loader):

        samples, predicates, joins, targets, sample_masks, predicate_masks, join_masks = data_batch

        samples, predicates, joins, targets = Variable(samples), Variable(predicates), Variable(joins), Variable(
            targets)            
        sample_masks, predicate_masks, join_masks = Variable(sample_masks), Variable(predicate_masks), Variable(
            join_masks)

        optimizer.zero_grad()
        outputs = model(samples, predicates, joins, sample_masks, predicate_masks, join_masks)
        loss = criterion(outputs, targets)
        loss_total += loss.item()
        loss.backward()
        optimizer.step()

    print("Epoch {}, loss: {}".format(epoch, loss_total / len(train_data_loader)))

RuntimeError: expected scalar type Long but found Float

In [ ]:
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    loss_total = 0.

    for batch_idx, data_batch in enumerate(train_data_loader):

        samples, predicates, joins, targets, sample_masks, predicate_masks, join_masks = data_batch

        samples, predicates, joins, targets = Variable(samples), Variable(predicates), Variable(joins), Variable(
            targets)            
        sample_masks, predicate_masks, join_masks = Variable(sample_masks), Variable(predicate_masks), Variable(
            join_masks)

        optimizer.zero_grad()
        outputs = model(samples, predicates, joins, sample_masks, predicate_masks, join_masks)
        loss = criterion(outputs, targets)
        loss_total += loss.item()
        loss.backward()
        optimizer.step()

    print("Epoch {}, loss: {}".format(epoch, loss_total / len(train_data_loader)))

RuntimeError: expected scalar type Long but found Float